<a href="https://colab.research.google.com/github/aniqaizuddin/Bugging-BadBad/blob/main/Bugging_Badbad_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fetch data for Random Forest

In [ ]:
!pip install cybotrade-datasource
import pandas as pd
from datetime import datetime, timezone
import asyncio
import cybotrade_datasource

API_KEY = "7FzTcgRWspHPdrSwl8N8yeW1jBbpCgp2isjiwFj4Hy0fpSgo"

async def fetch_data_from_endpoint(topic, start_time, end_time):
    data = await cybotrade_datasource.query_paginated(
        api_key=API_KEY,
        topic=topic,
        start_time=start_time,
        end_time=end_time
    )

    df = pd.DataFrame(data)
    if df.empty:
        return pd.DataFrame()

    if 'datetime' in df.columns:
        df = df.drop(columns=['datetime'])

    if 'v' in df.columns:
        col_name = topic.split('|')[-1].split('?')[0].replace('/', '_')
        df.rename(columns={'v': col_name}, inplace=True)

    return df

async def fetch_all_data():
    topics = [
        'cryptoquant|btc/market-data/price-ohlcv?window=hour',
        'cryptoquant|btc/exchange-flows/netflow?exchange=binance&window=hour',
        'glassnode|market/price_usd_close?a=BTC&i=1h',
        'glassnode|addresses/active_count?a=BTC&i=1h',
        'glassnode|indicators/sopr?a=BTC&i=1h',
        'glassnode|indicators/net_unrealized_profit_loss?a=BTC&i=1h'
    ]

    start_time = datetime(2020, 1, 1, tzinfo=timezone.utc)
    end_time = datetime(2025, 1, 1, tzinfo=timezone.utc)

    data_frames = await asyncio.gather(
        *[fetch_data_from_endpoint(topic, start_time, end_time) for topic in topics]
    )

    data_frames = [df for df in data_frames if not df.empty]

    random_forest_df = data_frames[0]
    for df in data_frames[1:]:
        random_forest_df = pd.merge(random_forest_df, df, on='start_time', how='outer')

    random_forest_df['date_time'] = pd.to_datetime(random_forest_df['start_time'], unit='ms', utc=True)
    cols = ['date_time'] + [col for col in random_forest_df.columns if col != 'date_time']
    random_forest_df = random_forest_df[cols]

    print(random_forest_df.head())
    return random_forest_df

# Run this in async environment
df = await fetch_all_data()

                  date_time     start_time        close         high  \
0 2020-01-01 00:00:00+00:00  1577836800000  7151.440796  7198.598589   
1 2020-01-01 01:00:00+00:00  1577840400000  7196.927148  7235.672799   
2 2020-01-01 02:00:00+00:00  1577844000000  7222.727164  7247.047404   
3 2020-01-01 03:00:00+00:00  1577847600000  7222.720666  7249.648983   
4 2020-01-01 04:00:00+00:00  1577851200000  7198.947366  7237.636658   

           low         open       volume  netflow_total  \
0  7150.499296  7193.015439  1672.404676      22.868824   
1  7149.239908  7150.443392  3118.882091      75.134734   
2  7189.056447  7218.593858  1843.909873     -99.358706   
3  7191.103017  7239.044622  1756.918684     -12.874796   
4  7189.931589  7221.203033  1155.153493      -1.265298   

   market_price_usd_close addresses_active_count  indicators_sopr  \
0             7180.453680                  31702         0.997009   
1             7204.251930                  27746         0.996879   
2    

Fetch data for HMM

In [ ]:
import pandas as pd
import asyncio
from datetime import datetime, timezone
import cybotrade_datasource

API_KEY = "7FzTcgRWspHPdrSwl8N8yeW1jBbpCgp2isjiwFj4Hy0fpSgo"

async def fetch_data_from_endpoint(topic, start_time, end_time):
    """
    Fetch data from a given endpoint and return as a DataFrame.
    """
    data = await cybotrade_datasource.query_paginated(
        api_key=API_KEY,
        topic=topic,
        start_time=start_time,
        end_time=end_time
    )

    # Convert the fetched data into a DataFrame
    df = pd.DataFrame(data)

        # Drop the original 'datetime' column if it exists
    if 'datetime' in df.columns:
        df = df.drop(columns=['datetime'])

    return df

async def fetch_all_data():
    """
    Fetch data from multiple endpoints efficiently and merge the results.
    """
    # Define the list of topics (endpoints) to fetch data from
    topics = [
        'cryptoquant|btc/market-data/price-ohlcv?window=hour',
        'cryptoquant|exchange-flows/netflow?exchange=binance&window=hour'
    ]

    # Set the time range for the query
    start_time = datetime(2020, 1, 1, tzinfo=timezone.utc)
    end_time = datetime(2025, 1, 1, tzinfo=timezone.utc)

    data_frames = await asyncio.gather(
        *[fetch_data_from_endpoint(topic, start_time, end_time) for topic in topics]
    )

    # Use asyncio.gather to fetch data from all endpoints concurrently
    data_frames = await asyncio.gather(
        *[fetch_data_from_endpoint(topic, start_time, end_time) for topic in topics]
    )

    # If needed, merge or further process the DataFrames here
    HMM_df = data_frames[0]  # Example of taking the first DataFrame
    for df in data_frames[1:]:
        HMM_df = pd.merge(HMM_df, df, on='start_time', how='outer')  # Adjust based on actual column names

    # Convert 'start_time' to human-readable datetime (UTC) and insert as a new column
    HMM_df['date_time'] = pd.to_datetime(HMM_df['start_time'], unit='ms', utc=True)

    # Optional: reorder columns to have 'datetime' first
    cols = ['date_time'] + [col for col in HMM_df.columns if col != 'date_time']
    HMM_df = HMM_df[cols]

    print(HMM_df.head())  # Display the first few rows of the merged DataFrame
    return HMM_df


# Running the fetch function
await fetch_all_data()

                  date_time     start_time        close         high  \
0 2020-01-01 00:00:00+00:00  1577836800000  7151.440796  7198.598589   
1 2020-01-01 01:00:00+00:00  1577840400000  7196.927148  7235.672799   
2 2020-01-01 02:00:00+00:00  1577844000000  7222.727164  7247.047404   
3 2020-01-01 03:00:00+00:00  1577847600000  7222.720666  7249.648983   
4 2020-01-01 04:00:00+00:00  1577851200000  7198.947366  7237.636658   

           low         open       volume  inflow_mean  inflow_mean_ma7  \
0  7150.499296  7193.015439  1672.404676     0.302603         1.137046   
1  7149.239908  7150.443392  3118.882091     0.666845         1.196020   
2  7189.056447  7218.593858  1843.909873     0.367091         1.030974   
3  7191.103017  7239.044622  1756.918684     0.626257         0.761248   
4  7189.931589  7221.203033  1155.153493     0.501650         0.759603   

   inflow_top10  inflow_total  outflow_mean  outflow_mean_ma7  outflow_top10  \
0     40.032496     59.007532      1.38995

,date_time,start_time,close,high,low,open,volume,inflow_mean,inflow_mean_ma7,inflow_top10,inflow_total,outflow_mean,outflow_mean_ma7,outflow_top10,outflow_total
0,2020-01-01 00:00:00+00:00,1577836800000,7151.440796,7198.598589,7150.499296,7193.015439,1672.404676,0.302603,1.137046,40.032496,59.007532,1.389950,15.662857,36.136796,36.138708
1,2020-01-01 01:00:00+00:00,1577840400000,7196.927148,7235.672799,7149.239908,7150.443392,3118.882091,0.666845,1.196020,63.786149,75.353517,0.109392,7.924586,0.218784,0.218784
2,2020-01-01 02:00:00+00:00,1577844000000,7222.727164,7247.047404,7189.056447,7218.593858,1843.909873,0.367091,1.030974,69.788983,87.367639,6.224212,8.409421,186.724062,186.726345
3,2020-01-01 03:00:00+00:00,1577847600000,7222.720666,7249.648983,7191.103017,7239.044622,1756.918684,0.626257,0.761248,65.126078,77.655807,11.316325,7.377755,90.530603,90.530603
4,2020-01-01 04:00:00+00:00,1577851200000,7198.947366,7237.636658,7189.931589,7221.203033,1155.153493,0.501650,0.759603,34.603547,42.640222,1.568054,7.200680,43.903559,43.905520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43843,2024-12-31 19:00:00+00:00,1735671600000,94085.636202,94264.179749,93592.749263,93754.906098,1037.504802,0.779300,3.106054,64.516826,65.461242,14.627777,9.098243,337.655549,351.066650
43844,2024-12-31 20:00:00+00:00,1735675200000,93591.540130,94152.510623,93303.882211,94031.718555,1942.061562,1.717087,3.262874,77.152187,77.268928,6.783539,6.787444,177.333078,183.155556
43845,2024-12-31 21:00:00+00:00,1735678800000,93721.025455,93960.663688,93322.009821,93496.089337,939.438137,0.622126,2.780909,26.650030,26.751399,5.859725,6.995488,214.996475,228.529295
43846,2024-12-31 22:00:00+00:00,1735682400000,93292.143937,93923.152711,93277.478941,93745.161839,584.460600,15.889100,4.908409,1111.197975,1144.015216,12.770989,7.693945,429.464474,446.984621
